# Ensemble Query Engine

When building a retrieval-augmented generation (RAG) application, experimenting with various query pipelines—such as top-k retrieval, keyword search, and knowledge graphs—is often necessary.

**Concept**: What if we could simultaneously use multiple strategies and have a language model (LLM):

1. Rate the relevance of each query result.

2. Synthesize the results into a coherent answer.

The Ensemble Query Engine allows you to do just that. This guide explains how to experiment with different query pipelines and strategies, have the LLM evaluate the relevance of each result, and synthesize the final response.

## Key Purposes

1. **Multi-Strategy Retrieval**: Try multiple retrieval strategies at once, such as top-k retrieval, keyword search, and knowledge graphs. This helps compare the effectiveness of various approaches.

2. **Relevance Evaluation**: Have the LLM rate how pertinent each retrieved result is to the original query, ensuring only the most relevant information is considered.

3. **Result Synthesis**: Let the LLM combine the most relevant information from different retrieval methods, leveraging its language understanding to create a comprehensive final answer.

## How to Use the Ensemble Query Engine

1. **Set Up Retrieval Tools**: Configure different retrieval tools, such as a keyword search tool and a vector search tool.

2. **Configure the Router Query Engine**: Set up a router query engine with a selector to choose the relevant retrievals and a summarizer to synthesize the final answer.

3. **Run Queries**: Use the router query engine to process queries and return synthesized responses that leverage multiple retrieval strategies evaluated by the LLM.

## Benefits

The Ensemble Query Engine enables you to harness the strengths of different retrieval methods and the reasoning capabilities of LLMs in a unified querying interface. It simplifies experimentation with various approaches, helping you find the optimal configuration for your RAG application.

In [ ]:
%%capture
!pip install llama-index==0.10.37 llama-index-embeddings-openai==0.1.9 qdrant-client==1.9.1 llama-index-vector-stores-qdrant==0.2.8 llama-index-llms-openai==0.1.19

In [1]:
import os
import sys
from getpass import getpass
import nest_asyncio

from IPython.display import Markdown, display

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv("")

sys.path.append('../helpers')

from utils import setup_llm, setup_embed_model, setup_vector_store

In [2]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OpenAI API key: ")

In [3]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")

In [4]:
# QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

QDRANT_URL=":memory:"

In [5]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

In [6]:
from llama_index.core.settings import Settings
from utils import setup_llm, setup_embed_model

setup_llm(
    provider="openai",
    api_key=OPENAI_API_KEY, 
    model="gpt-4o", 
    temperature=0.75, 
    system_prompt="""Use ONLY the provided context and generate a complete, coherent answer to the user's query. 
    Your response must be grounded in the provided context and relevant to the essence of the user's query.
    """
    )

setup_embed_model(
    provider="openai", 
    model="text-embedding-3-small",
    api_key=OPENAI_API_KEY
    )

In [7]:
import random
from llama_index.core.storage.docstore import SimpleDocumentStore
from utils import get_documents_from_docstore, group_documents_by_author, sample_documents

documents = get_documents_from_docstore("../data/words-of-the-senpais")

random.seed(42)

documents_by_author = group_documents_by_author(documents)

senpai_documents = sample_documents(documents_by_author, num_samples=10)

In [8]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=128, chunk_overlap=8)

nodes = splitter.get_nodes_from_documents(senpai_documents)

In [9]:
from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults()

storage_context.docstore.add_documents(nodes)

### [`SimpleKeywordTableIndex`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/indices/keyword_table/simple_base.py) Class Overview

The `SimpleKeywordTableIndex` class is a simplified version of a keyword-based indexing system. 

During index construction, the KeywordTableIndex takes a dataset of text documents, chunks them, and uses GPT to extract relevant keywords. These keywords are stored in a table referencing the respective text chunk. 

During a query, the KeywordTableIndex extracts relevant keywords and uses them to retrieve a set of candidate text chunk IDs. The initial answer is constructed using the first text chunk and then refined with subsequent chunks.

- **Index Construction**: 
    - Splits text documents into chunks.
    - Extracts keywords for each chunk.
    - Stores keywords in a table referencing the text chunks.

- **Query Modes**:
    - **Default**: Uses GPT for keyword extraction and constructs answers by refining text chunks.
    - **Simple**: Uses regex for keyword extraction (implemented by `SimpleKeywordTableIndex`).


In [10]:
from llama_index.core import SimpleKeywordTableIndex, VectorStoreIndex

keyword_index = SimpleKeywordTableIndex(
    nodes,
    storage_context=storage_context,
    show_progress=True,
)
vector_index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    show_progress=True,
)

Extracting keywords from nodes:   0%|          | 0/259 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/259 [00:00<?, ?it/s]

In [11]:
from llama_index.core import PromptTemplate

QA_PROMPT_TMPL = """
Context:
---------------------
{context_str}
---------------------
Based on the context above, answer the question below. If the answer is not in the context, 
inform the user without making up an answer. Additionally, provide a relevance score for the answer.

Question: {query_str}
Answer (with relevance score):
"""


QA_PROMPT = PromptTemplate(QA_PROMPT_TMPL)

keyword_query_engine = keyword_index.as_query_engine(
    text_qa_template=QA_PROMPT
)

vector_query_engine = vector_index.as_query_engine(text_qa_template=QA_PROMPT)

In [12]:
response = vector_query_engine.query(
    "What is the importance of focusing on who you work with and what you work on, rather than just how hard you work?"
)

In [13]:
print(response)

The importance of focusing on who you work with and what you work on, rather than just how hard you work, is emphasized by Naval Ravikant as being more crucial than the amount of effort you put in. The context highlights that aligning yourself with the right people and the right projects can have a greater impact on your success than merely working hard. This includes leveraging your specific knowledge, accountability, and authentic skill set to position yourself to be the best in the world at what you do.

Relevance score: 9/10


In [14]:
response = keyword_query_engine.query(
    "What is the importance of focusing on who you work with and what you work on, rather than just how hard you work?"
)

In [15]:
print(response)

The importance of focusing on who you work with and what you work on, rather than just how hard you work, is highlighted in the context from "The Almanack of Naval Ravikant" by Naval Ravikant. The excerpt suggests that while hard work is necessary, the impact of your work is significantly influenced by the people you collaborate with and the projects you choose to engage in. This implies that strategic decisions about partnerships and project selection can lead to more meaningful and efficient outcomes, as opposed to merely increasing the amount of effort you put in.

Relevance Score: 10/10


### [QueryEngineTool](https://github.com/run-llama/llama_index/blob/7849b1a851d88ee28e1bfd05d19f18e40d5b8e10/llama-index-core/llama_index/core/tools/query_engine.py#L17)

Tools are abstractions designed to be used by data agents or LLMs and provide a structured way for them to perform tasks

A `QueryEngineTool` is a specific type of tool designed to interface with and wrap existing query engines. It enables agents to perform complex queries by leveraging the capabilities of the underlying query engine.

#### Use Cases

- **Integrating Query Engines**: Allows agents to interact with query engines and other agents.

- **Complex Query Handling**: Helps execute sophisticated queries and data retrieval operations.

In [16]:
from llama_index.core.tools import QueryEngineTool

keyword_tool = QueryEngineTool.from_defaults(
    query_engine=keyword_query_engine,
    description="Useful for answering finding documents based on keywords and incomplete thoughts from a user.",
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for answering fully-formed questions from a user.",
)

## Define a Router Query Engine

The [`LLMMultiSelector`](https://github.com/run-llama/llama_index/blob/7849b1a851d88ee28e1bfd05d19f18e40d5b8e10/llama-index-core/llama_index/core/selectors/llm_selectors.py#L141) uses LLMs to make decisions. Uses a prompt to present choices to the LLM, which then selects the most relevant options based on the query. It can be used alone or integrated into query engines and retrievers. 

#### Key Functions:

1. **Data Source Selection**: Chooses the best data source from multiple options.

2. **Operational Decisions**: Decides whether to perform summarization or semantic search.

3. **Multi-Routing**: Evaluates multiple choices simultaneously and combines the results.


### Use Cases

- Selecting the right data source.
- Choosing between summarization and semantic search.
- Combining results from multiple choices.


In [17]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMMultiSelector

from llama_index.core.response_synthesizers import TreeSummarize

TREE_SUMMARIZE_PROMPT_TMPL = """
Context from multiple sources is below. Each source may have a relevance score.

---------------------
{context_str}
---------------------

Based on the information from the sources above, answer the question below. 

If the answer is not in the context, inform the user without making up an answer.

Question: {query_str}
Answer:
"""

tree_summarize = TreeSummarize(
    summary_template=PromptTemplate(TREE_SUMMARIZE_PROMPT_TMPL)
)

query_engine = RouterQueryEngine(
    selector=LLMMultiSelector.from_defaults(),
    query_engine_tools=[
        keyword_tool,
        vector_tool,
    ],
    summarizer=tree_summarize,
    verbose=True,
)

In [18]:
response = await query_engine.aquery(
    "How can I develop specific knowledge that will help me build wealth and achieve happiness?"
)
print(response)

Selecting query engine 1: Useful for answering fully-formed questions from a user..
The context provided does not specifically address how to develop specific knowledge that will help build wealth and achieve happiness. It mentions the importance of avoiding ruin and making better decisions, but does not provide detailed strategies or steps for developing specific knowledge.

Relevance score: 4/10


In [19]:
from llama_index.core.response.notebook_utils import display_response

display_response(
    response, show_source=True, source_length=500, show_source_metadata=True
)

**`Final Response:`** The context provided does not specifically address how to develop specific knowledge that will help build wealth and achieve happiness. It mentions the importance of avoiding ruin and making better decisions, but does not provide detailed strategies or steps for developing specific knowledge.

Relevance score: 4/10

---

**`Source Node 1/2`**

**Node ID:** 2727d142-9426-4d52-9bfd-44116a286d15<br>**Similarity:** 0.43822466974483665<br>**Text:** technology and large workforces and capital, our decisions are leveraged more and more. If you can be more right and more rational, youre going to get nonlinear returns in your life. I love the blog Farnam Street because it really focuses on helping you be more accurate, an overall better decision-maker. Decision-making is everything. The more you know, the less you diversify.<br>**Metadata:** {'page_number': 76, 'file_name': '../data/almanack_of_naval_ravikant.pdf', 'title': 'The Almanack of Naval Ravikant', 'author': 'Naval Ravikant'}<br>

---

**`Source Node 2/2`**

**Node ID:** 577b4652-2f57-49bd-8b66-a1db03f5e355<br>**Similarity:** 0.4298802828072778<br>**Text:** The combination of those over a long period of time with the magic of compound interest will make you wealthy. The one thing you have to avoid is the risk of ruin. Avoiding ruin means stay out of jail. So, dont do anything ille-<br>**Metadata:** {'page_number': 36, 'file_name': '../data/almanack_of_naval_ravikant.pdf', 'title': 'The Almanack of Naval Ravikant', 'author': 'Naval Ravikant'}<br>

In [20]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='2727d142-9426-4d52-9bfd-44116a286d15', embedding=None, metadata={'page_number': 76, 'file_name': '../data/almanack_of_naval_ravikant.pdf', 'title': 'The Almanack of Naval Ravikant', 'author': 'Naval Ravikant'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='789263af-c3cb-4204-9a25-69d886e33b73', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_number': 76, 'file_name': '../data/almanack_of_naval_ravikant.pdf', 'title': 'The Almanack of Naval Ravikant', 'author': 'Naval Ravikant'}, hash='3a49e6f7304356b26fde4a4bdd5ca2747beef954f087319476965d27832e485c'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='bb5fcb9c-11fe-4f6e-b74b-8d5c02c66c12', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='2c7fb28a5b690ce69c33842bf7c7f83eb7164010e9a14f92d362072b45aff070')}, text='technology and large workforces and capital, our decisions are leveraged more and more. If

In [23]:
response = await query_engine.aquery(
    "specific knowledge, build wealth, achieve happiness?"
)

display_response(
    response, show_source=True, source_length=500, show_source_metadata=True
)

Selecting query engine 0: The query 'specific knowledge, build wealth, achieve happiness?' consists of keywords and incomplete thoughts rather than a fully-formed question..


**`Final Response:`** The provided context does not explicitly address how to build wealth or achieve happiness through specific knowledge. However, there are relevant insights from the excerpts:

1. **Building Wealth through Innovation and Risk-taking:**
   - From "The Anthology of Balaji Srinivasan," it emphasizes the golden age for builders, leveraging open source, 3D printing, app stores, and crowdfunding to de-risk, prototype, and accept global payments (page 151). This highlights that innovation and taking risks can be pathways to wealth.

2. **Self-Knowledge and Happiness:**
   - From "Striking Thoughts" by Bruce Lee, it discusses self-knowledge as the road to freedom and emphasizes understanding oneself from moment to moment (page 174). This suggests that self-awareness and inner understanding are crucial for achieving personal happiness.

Relevance Score: 7/10
- While the context provides insights into building wealth through innovation and achieving happiness through self-knowledge, it does not provide a comprehensive answer connecting all three aspects (specific knowledge, building wealth, achieving happiness) directly.

If you have more specific questions or need further details, feel free to ask!

---

**`Source Node 1/6`**

**Node ID:** 87d615bd-6676-4c47-8bb6-a6556f7cfc0b<br>**Similarity:** None<br>**Text:** Thats not a theoretical example. In China, theyre building skyscrapers in two weeks flat. You can see it on YouTube. They put up a camera and build twenty-four hours a day from prefab material assembled on site. Theyre building 100x faster than you can build in the US. In the US, it took ten years to rebuild the World Trade Center.<br>**Metadata:** {'page_number': 9, 'file_name': '../data/anthology_of_balaji.pdf', 'title': 'The Anthology of Balaji Srinivasan', 'author': 'Balaji Srinivasan'}<br>

...

**`Source Node 5/6`**

**Node ID:** 475faf7e-5649-44b4-81bf-a847104492fb<br>**Similarity:** None<br>**Text:** the branches. - What we are after is the ROOT and not the branches. The root is the real knowledge; the branches are surface knowledge. Real knowledge breeds "body feel" and personal expression; surface knowledge breeds mechanical conditioning and imposing limitation and squelches creativity. Express your total presence from the root. - Be at once absorbingly open and rootily relaying your captivating total presence with appropriate inward time. The root is the starting point.<br>**Metadata:** {'page_number': 16, 'file_name': '../data/striking-thoughts.pdf', 'title': 'Striking Thoughts', 'author': 'Bruce Lee'}<br>

---

**`Source Node 6/6`**

**Node ID:** 28b6ed5e-737b-487c-8e6d-11e428d23ad1<br>**Similarity:** None<br>**Text:** Was it that Europeans are somehow racially superior? Was it their religion? The answer (or at least the proximate cause) may be that the Europeans rode on the crest of a powerful new idea: allowing those who made a lot of money to keep it. Once youre allowed to do that, people who want to get rich can do it by generating wealth instead of stealing it. The resulting technological growth translates not only into wealth but into military power.<br>**Metadata:** {'page_number': 107, 'file_name': '../data/hackers_and_painters.pdf', 'title': 'Hackers and Painters', 'author': 'Paul Graham'}<br>

In [24]:
response = await query_engine.aquery(
    "Calm mind, clear schedule, clear mind. What do I need to do to achieve these?"
)
display_response(
    response, show_source=True, source_length=500, show_source_metadata=True
)

Selecting query engine 1: The user's query is a fully-formed question seeking advice on achieving a calm mind and clear schedule..


**`Final Response:`** Based on the provided context, to achieve a calm mind, clear schedule, and clear mind, you need to:

1. Emancipate your mind from old habits, prejudices, restrictive thought processes, and even ordinary thought itself (Striking Thoughts, Bruce Lee).
2. Cultivate an alert mind, which involves being sincere to oneself and staying serious and sincere in your pursuits (Striking Thoughts, Bruce Lee).
3. Avoid letting your preconceived notions cloud your reality, particularly when mixing different aspects of your life such as politics and business (The Almanack of Naval Ravikant, Naval Ravikant).

Relevance Score: 9/10

---

**`Source Node 1/2`**

**Node ID:** 3d9b728d-4c72-4a78-981d-be905d632eb4<br>**Similarity:** 0.3515060115569545<br>**Text:** - [To] bring the mind into sharp focus and to make it alert so that it can immediately intuit truth, which is everywhere, the mind must be emancipated from old habits, prejudices, restrictive thought process, and even ordinary thought itself. Cultivate an alert mind. - To be on the alert means to be deadly serious; to be deadly serious means to be sincere to oneself, and it is sincerity that finally leads to the Way.<br>**Metadata:** {'page_number': 43, 'file_name': '../data/striking-thoughts.pdf', 'title': 'Striking Thoughts', 'author': 'Bruce Lee'}<br>

---

**`Source Node 2/2`**

**Node ID:** 74876a11-0f78-412b-94af-57d7d2bdfed2<br>**Similarity:** 0.3452401383699676<br>**Text:** The monkey mind will always respond with this regurgitated emotional response to what it thinks the world should be. Those desires will cloud your reality. This happens a lot of times when people are mixing politics and business. The number one thing clouding us from being able to see reality is we have preconceived notions of the way it should be.<br>**Metadata:** {'page_number': 66, 'file_name': '../data/almanack_of_naval_ravikant.pdf', 'title': 'The Almanack of Naval Ravikant', 'author': 'Naval Ravikant'}<br>